In [1]:
import os,sys
import pickle
import argparse
from sklearn.metrics import confusion_matrix
from datetime import datetime

from vrae.vrae import VRAE, VRAEC
from vrae.utils import *
import numpy as np
import torch
from torch.utils import data as data2
from torch.utils.data.dataset import random_split
from torch.utils.data import DataLoader, TensorDataset
from datetime import datetime
import matplotlib.pyplot as plt

In [6]:

class DatasetSq(data2.Dataset):
    def __init__(self, data_dir, list_IDs, labels, cut_idx=None):
        # initialize
        self.data_dir = data_dir
        self.labels = labels
        self.list_IDs = list_IDs
        self.cut_idx = cut_idx
    def __len__(self):
        'Denotes the total number of samples'
        return len(self.list_IDs)
    
    def __getitem__(self, index):
        'Generates one sample of data'
        # Select sample
        ID = self.list_IDs[index]
        # Load data and get label
        X = torch.load(self.data_dir + ID + '.pt')
        X.transpose_(0, 2) # for Icub
        X.unsqueeze_(0)
        
        if self.cut_idx is None:
            # use original len
            self.cut_idx = X.size()[1]
        
        X = X[:,:self.cut_idx,:] 
        X = X.contiguous().view(X.size(1),-1)    
        # print("X", X.size())   
        y = self.labels[ID]

        return X, y
    
    def get_X(self):
        X = []
        for i in range(len(self.list_IDs)):
            ID = self.list_IDs[i]
            x = torch.load(self.data_dir + ID + '.pt')
            x.unsqueeze_(0)

            if self.cut_idx is None:
                # use original len
                self.cut_idx = x.size()[1]
            
            x = x[:,:self.cut_idx,:]
    
            # change for vae only
            x = x.view(x.size(1),-1)       
            
            X.append(x.numpy())
        return np.array(X)
    
    def get_y(self):
        y = []
        for i in range(len(self.list_IDs)):
            ID = self.list_IDs[i]
            y.append(self.labels[ID])
        return np.array(y)


In [2]:
dload = './model_dir' #download directory
model_name = 'Icub_CNN.pth'

In [3]:
num_class = 20
hidden_size = 90
hidden_layer_depth = 1
latent_length = 40
batch_size = 32
learning_rate = 0.0005
n_epochs = 15000
dropout_rate = 0.2
optimizer = 'Adam' # options: ADAM, SGD
cuda = True # options: True, False
print_every=30
clip = True # options: True, False
max_grad_norm=5
loss = 'MSELoss' # options: SmoothL1Loss, MSELoss
block = 'LSTM' # options: LSTM, GRU
header = "CNN"
w_r = 0.001
w_k = 1
w_c = 10
sequence_length = 75
number_of_features = 60

In [4]:
vraec = VRAEC(num_class=num_class,
            sequence_length=sequence_length,
            number_of_features = number_of_features,
            hidden_size = hidden_size, 
            hidden_layer_depth = hidden_layer_depth,
            latent_length = latent_length,
            batch_size = batch_size,
            learning_rate = learning_rate,
            n_epochs = n_epochs,
            dropout_rate = dropout_rate,
            optimizer = optimizer, 
            cuda = cuda,
            print_every=print_every, 
            clip=clip, 
            max_grad_norm=max_grad_norm,
            loss = loss,
            block = block,
            dload = dload,
            model_name=model_name,
            header=header,
            w_r = 0.001, 
            w_k = 1, 
            w_c = 10)


loss weights for vraec w_r: 0.001, w_k: 1, w_c: 10


/home/students/student6_16/anaconda3/lib/python3.7/site-packages/torch/nn/modules/rnn.py:51: UserWarning:

dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1

/home/students/student6_16/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:43: UserWarning:

size_average and reduce args will be deprecated, please use reduction='sum' instead.



In [8]:
# Load data
datafile = 'Icub.pkl'
data_dir = 'data/Icub/'

# [train_ids, train_labels,  valid_ids, valid_labels, test_ids, test_labels] = pickle.load(open(datafile, 'rb')) # for xx_20_50.pkl
[train_ids, train_labels,  valid_ids, valid_labels] = pickle.load(open(datafile, 'rb')) # for xx.pkl
train_dataset = DatasetSq(data_dir, train_ids, train_labels, cut_idx=sequence_length)
valid_dataset = DatasetSq(data_dir, valid_ids, valid_labels, cut_idx=sequence_length)
# X_train = train_dataset.get_X()
y_train = train_dataset.get_y()
# X_val = valid_dataset.get_X()
y_val = valid_dataset.get_y()

In [10]:
vraec.load(dload + '/' +model_name)

FileNotFoundError: [Errno 2] No such file or directory: 'Icub_CNN.pth'